In [1]:
from importlib import reload
import sys
from imp import reload
import warnings
warnings.filterwarnings('ignore')
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [6]:
import pandas as pd
df = pd.read_csv('training_data.csv')
#df = df.drop(['id','qid1','qid2'],axis=1)
df.head()

,Job_Description,resume_Description,Label
0,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"We provide telemarketing services, appointment...",0
1,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"25 Years exp, over 3 million accident and tick...",1
2,CDL Driver (Enumclaw) <image 1 of 1>compensati...,Bilingual Live-in caregiver available NOW for ...,0
3,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"RACHEL DAWSON Seattle, WA 98118OBJECTIVE Quick...",0
4,CDL Driver (Enumclaw) <image 1 of 1>compensati...,I am 35 year old male who lives on the Eastsid...,1


In [3]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    #text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [7]:
df['Job'] = df.Job_Description.apply(lambda x: clean_text(str(x)))
df.head()

,Job_Description,resume_Description,Label,Job
0,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"We provide telemarketing services, appointment...",0,cdl driver enumclaw image 1 of 1compensation d...
1,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"25 Years exp, over 3 million accident and tick...",1,cdl driver enumclaw image 1 of 1compensation d...
2,CDL Driver (Enumclaw) <image 1 of 1>compensati...,Bilingual Live-in caregiver available NOW for ...,0,cdl driver enumclaw image 1 of 1compensation d...
3,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"RACHEL DAWSON Seattle, WA 98118OBJECTIVE Quick...",0,cdl driver enumclaw image 1 of 1compensation d...
4,CDL Driver (Enumclaw) <image 1 of 1>compensati...,I am 35 year old male who lives on the Eastsid...,1,cdl driver enumclaw image 1 of 1compensation d...


In [8]:
df['resume'] = df.resume_Description.apply(lambda x: clean_text(str(x)))
df.head()

,Job_Description,resume_Description,Label,Job,resume
0,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"We provide telemarketing services, appointment...",0,cdl driver enumclaw image 1 of 1compensation d...,we provide telemarketing service appointment s...
1,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"25 Years exp, over 3 million accident and tick...",1,cdl driver enumclaw image 1 of 1compensation d...,25 year exp over 3 million accident and ticket...
2,CDL Driver (Enumclaw) <image 1 of 1>compensati...,Bilingual Live-in caregiver available NOW for ...,0,cdl driver enumclaw image 1 of 1compensation d...,bilingual livein caregiver available now for p...
3,CDL Driver (Enumclaw) <image 1 of 1>compensati...,"RACHEL DAWSON Seattle, WA 98118OBJECTIVE Quick...",0,cdl driver enumclaw image 1 of 1compensation d...,rachel dawson seattle wa 98118objective quick ...
4,CDL Driver (Enumclaw) <image 1 of 1>compensati...,I am 35 year old male who lives on the Eastsid...,1,cdl driver enumclaw image 1 of 1compensation d...,i be 35 year old male who life on the eastside...


In [9]:
df_final = df[['Job','resume','Label']]
df_final.head()

,Job,resume,Label
0,cdl driver enumclaw image 1 of 1compensation d...,we provide telemarketing service appointment s...,0
1,cdl driver enumclaw image 1 of 1compensation d...,25 year exp over 3 million accident and ticket...,1
2,cdl driver enumclaw image 1 of 1compensation d...,bilingual livein caregiver available now for p...,0
3,cdl driver enumclaw image 1 of 1compensation d...,rachel dawson seattle wa 98118objective quick ...,0
4,cdl driver enumclaw image 1 of 1compensation d...,i be 35 year old male who life on the eastside...,1


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , CuDNNLSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, concatenate, dot
from keras.models import Model, Sequential

In [11]:
total_text = pd.concat([df['Job'], df['resume']]).reset_index(drop=True)
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(total_text)
question_1_sequenced = tokenizer.texts_to_sequences(df['Job'])
question_2_sequenced = tokenizer.texts_to_sequences(df['resume'])

In [12]:
maxlen = 500
question_1_padded = pad_sequences(question_1_sequenced, maxlen=maxlen)
question_2_padded = pad_sequences(question_2_sequenced, maxlen=maxlen)

In [13]:
y = df['Label']

In [15]:
embedding_size = 128

inp1 = Input(shape=(500,))
inp2 = Input(shape=(500,))

x1 = Embedding(max_features, embedding_size)(inp1)
x2 = Embedding(max_features, embedding_size)(inp2)

x3 = Bidirectional(CuDNNLSTM(32, return_sequences = True))(x1)
x4 = Bidirectional(CuDNNLSTM(32, return_sequences = True))(x2)

x5 = GlobalMaxPool1D()(x3)
x6 = GlobalMaxPool1D()(x4)

x7 =  dot([x5, x6], axes=1)

x8 = Dense(40, activation='relu')(x7)
x9 = Dropout(0.05)(x8)
x10 = Dense(10, activation='relu')(x9)
output = Dense(1, activation="sigmoid")(x10)

model = Model(inputs=[inp1, inp2], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 100
epochs = 3
model.fit([question_1_padded, question_2_padded], y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/3


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by {{node model_1/bidirectional_3/forward_cu_dnnlstm_3/CudnnRNNV2}} with these attrs: [seed=0, dropout=0, input_mode="linear_input", T=DT_FLOAT, direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true]
Registered devices: [CPU]
Registered kernels:
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]

	 [[model_1/bidirectional_3/forward_cu_dnnlstm_3/CudnnRNNV2]] [Op:__inference_train_function_5622]